In [ ]:
import firebase_admin
from firebase_admin import credentials, storage
from tqdm.notebook import tqdm
import os

SERVICE_ACCOUNT_KEY = "rok-stats-sa.json"

BUCKET_NAME = "rok-stats-61b0d.firebasestorage.app"

class TqdmStream:
    def __init__(self, file_obj, pbar):
        self.file_obj = file_obj
        self.pbar = pbar

    def read(self, size=-1):
        data = self.file_obj.read(size)
        self.pbar.update(len(data))
        return data

    def seek(self, offset, whence=0):
        return self.file_obj.seek(offset, whence)

    def tell(self):
        return self.file_obj.tell()

    def close(self):
        return self.file_obj.close()


def upload_to_firebase(local_file_path, destination_blob_name):
    # Initialize app if not already
    if not firebase_admin._apps:
        cred = credentials.Certificate(SERVICE_ACCOUNT_KEY)
        firebase_admin.initialize_app(cred, {"storageBucket": BUCKET_NAME})

    bucket = storage.bucket()
    blob = bucket.blob(destination_blob_name)

    file_size = os.path.getsize(local_file_path)

    with open(local_file_path, "rb") as f, tqdm(
        total=file_size, unit="B", unit_scale=True, desc="Uploading"
    ) as pbar:
        wrapped_file = TqdmStream(f, pbar)
        blob.upload_from_file(wrapped_file)

    # Make public (optional)
    blob.make_public()
    print(f"\n✅ Upload complete: {blob.public_url}")

def download_from_firebase(destination_blob_name, local_file_path, chunk_size=1024 * 1024):
    """Downloads a file from Firebase Storage with progress bar."""
    if not firebase_admin._apps:
        cred = credentials.Certificate(SERVICE_ACCOUNT_KEY)
        firebase_admin.initialize_app(cred, {"storageBucket": BUCKET_NAME})

    bucket = storage.bucket()
    blob = bucket.blob(destination_blob_name)

    file_size = blob.size  # size in bytes (metadata)
    if file_size is None:
        blob.reload()  # force load metadata
        file_size = blob.size

    with open(local_file_path, "wb") as f, tqdm(
        total=file_size, unit="B", unit_scale=True, desc="Downloading"
    ) as pbar:
        start = 0
        while start < file_size:
            end = min(start + chunk_size, file_size) - 1
            chunk = blob.download_as_bytes(start=start, end=end)
            f.write(chunk)
            pbar.update(len(chunk))
            start += len(chunk)

    print(f"\n✅ Download complete: {local_file_path}")

In [2]:
# upload_to_firebase("/home/marco_dossena/PHD/KGEmbeddings/models/TransE_umls_0/relation_embedding.npy", "models/TransE_umls_0/relation_embedding.npy")
# upload_to_firebase("/home/marco_dossena/PHD/KGEmbeddings/models/TransE_umls_0/entity_embedding.npy", "models/TransE_umls_0/entity_embedding.npy")

In [3]:
download_from_firebase("models/TransE_umls_0/relation_embedding.npy", "downloaded_relation_embedding.npy")

Downloading:   0%|          | 0.00/897k [00:00<?, ?B/s]


✅ Download complete: downloaded_relation_embedding.npy
